# Import Libraries

In [1]:
import optuna
import os
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMClassifier
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [2]:
def read_file(*args,file_type="csv")->pd.DataFrame:
    """:parameter input directories in sequence
        :return DataFrame"""
    
    path=os.path.join(*args)
    if file_type == "tsv":
        return pd.read_csv(path, sep="\t")
    return pd.read_csv(path)

# Experiments

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [4]:
mlflow.set_experiment("DifferentModelsExperiments2")

<Experiment: artifact_location='/home/spynom/mlruns/14', creation_time=1729323387786, experiment_id='14', last_update_time=1729323387786, lifecycle_stage='active', name='DifferentModelsExperiments2', tags={}>

In [5]:
df=read_file("..","data","processed","cleaned.csv").dropna(how="any").drop_duplicates()

X=df.comment
y=df.category+1

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
vectorizer.fit(X)  # Fit on all vocab data
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)  # Transform test data

In [6]:

class_weights = compute_class_weight('balanced', classes=np.array([0, 1, 2]), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

In [9]:
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_BalanceWeight_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        train_y_pred=model.predict(X_train)
        test_y_pred = model.predict(X_test)

        # Log accuracy
        train_accuracy = accuracy_score(y_train,train_y_pred)
        test_accuracy = accuracy_score(y_test, test_y_pred)
        
        mlflow.log_metric("train_accuracy", train_accuracy)
        mlflow.log_metric("test_accuracy", test_accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, test_y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [10]:
# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42,class_weight=class_weights_dict)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for LightGBM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42,class_weight=class_weights_dict)

    # Log the best model with MLflow, passing the algo_name as "LightGBM"
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for LightGBM
run_optuna_experiment()

[I 2024-10-19 13:14:25,188] A new study created in memory with name: no-name-a5635b10-26db-4693-9b36-86a43be2a134


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.755771 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 130918
[LightGBM] [Info] Number of data points in the train set: 28992, number of used features: 4418
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


[I 2024-10-19 13:14:54,776] Trial 0 finished with value: 0.6370033112582781 and parameters: {'n_estimators': 286, 'learning_rate': 0.001889661700927994, 'max_depth': 10}. Best is trial 0 with value: 0.6370033112582781.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.633151 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 130918
[LightGBM] [Info] Number of data points in the train set: 28992, number of used features: 4418
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

[I 2024-10-19 13:15:04,097] Trial 1 finished with value: 0.5281456953642384 and parameters: {'n_estimators': 216, 'learning_rate': 0.00011892189705799174, 'max_depth': 4}. Best is trial 0 with value: 0.6370033112582781.


...

[I 2024-10-19 13:22:42,954] Trial 28 finished with value: 0.7141280353200883 and parameters: {'n_estimators': 78, 'learning_rate': 0.03965608951879966, 'max_depth': 10}. Best is trial 17 with value: 0.8064293598233996.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.503872 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 130918
[LightGBM] [Info] Number of data points in the train set: 28992, number of used features: 4418
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

[I 2024-10-19 13:22:53,723] Trial 29 finished with value: 0.7941501103752759 and parameters: {'n_estimators': 187, 'learning_rate': 0.05971602490898236, 'max_depth': 10}. Best is trial 17 with value: 0.8064293598233996.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.415847 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 130918
[LightGBM] [Info] Number of data points in the train set: 28992, number of used features: 4418
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

2024/10/19 13:23:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/19 13:23:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM_BalanceWeight_TFIDF_Trigrams at: http://127.0.0.1:5000/#/experiments/14/runs/90bd98917a584c2992fad5c2442ab659.
2024/10/19 13:23:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/14.
